<a href="https://colab.research.google.com/github/nataliakotsani/ASMA/blob/main/ASMA_Tools_Natalia_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Habitual Frequency

1. Select Button: Choose Files
2. Select a file with a filename "F0.wav"

In [19]:
# Assistance for students in Singing and Music Aesthetics (ASMA) 2022
# Version 2
# Research Team:
# Anastasia Georgaki (Coordinator)
# Areti Andreopoulou, Iakovos Steinhauer, Vangelis Angelakis, Sofia Stavropoulou, Natalia Kotsani, Giorgos Dedousis, Kostas Katsantonis
# Author: Natalia Kotsani

!pip install praat-parselmouth
import parselmouth
import parselmouth
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import struct
from scipy import signal as sg

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file'.format(
      name=fn, length=len(uploaded[fn])))

sns.set()
snd = parselmouth.Sound('F0.wav')

def draw_spectrogram(spectrogram, dynamic_range=70):
    X, Y = spectrogram.x_grid(), spectrogram.y_grid()
    sg_db = 10 * np.log10(spectrogram.values)
    plt.pcolormesh(X, Y, sg_db, vmin=sg_db.max() - dynamic_range, cmap='afmhot')
    plt.ylim([spectrogram.ymin, spectrogram.ymax])
    plt.xlabel("time [s]")
    plt.ylabel("frequency [Hz]")

def draw_pitch(pitch):
    pitch_values = pitch.selected_array['frequency']
    pitch_values[pitch_values==0] = np.nan
    plt.plot(pitch.xs(), pitch_values, 'o', markersize=5, color='w')
    plt.plot(pitch.xs(), pitch_values, 'o', markersize=2)
    plt.grid(False)
    plt.ylim(0, pitch.ceiling)
    plt.ylabel("fundamental frequency [Hz]")

pitch = snd.to_pitch()
pre_emphasized_snd = snd.copy()
pre_emphasized_snd.pre_emphasize()
spectrogram = pre_emphasized_snd.to_spectrogram(window_length=0.03, maximum_frequency=8000)
plt.figure()
draw_spectrogram(spectrogram)
plt.twinx()
draw_pitch(pitch)
plt.xlim([snd.xmin, snd.xmax])
plt.savefig('f0.png')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


MessageError: ignored

# Vowel Map

1. Select Button: Choose Files
2. Select a file with a filename "A1.wav", "A2.wav", ...

In [ ]:
# Assistance for students in Singing and Music Aesthetics (ASMA) 2022
# Version 2
# Research Team:
# Anastasia Georgaki (Coordinator)
# Areti Andreopoulou, Iakovos Steinhauer, Vangelis Angelakis, Sofia Stavropoulou, Natalia Kotsani, Giorgos Dedousis, Kostas Katsantonis
# Author: Natalia Kotsani

!pip install praat-parselmouth
import parselmouth
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import struct
from scipy import signal as sg
from parselmouth.praat import call
import statistics
import os
from pathlib import Path
from google.colab import files

def measureFormants(f0min, f0max, p):
    snd = parselmouth.Sound(p)
    pitch = call(snd, "To Pitch (cc)", 0, f0min, 15, 'no', 0.03, 0.45, 0.01, 0.35, 0.14, f0max)
    pointProcess = call(snd, "To PointProcess (periodic, cc)", f0min, f0max)

    formants = call(snd, "To Formant (burg)", 0.1, 5, 5500, 0.025, 30)
    numPoints = call(pointProcess, "Get number of points")

    f1_list = []
    f2_list = []
    f3_list = []
    f4_list = []

    for point in range(0, numPoints):
        point += 1
        t = call(pointProcess, "Get time from index", point)
        f1 = call(formants, "Get value at time", 1, t, 'Hertz', 'Linear')
        f2 = call(formants, "Get value at time", 2, t, 'Hertz', 'Linear')
        f1_list.append(f1)
        f2_list.append(f2)
    
    f1_list = [f1 for f1 in f1_list if str(f1) != 'nan']
    f2_list = [f2 for f2 in f2_list if str(f2) != 'nan']

    f1_mean = statistics.mean(f1_list)
    f2_mean = statistics.mean(f2_list)

    return (f1_mean, f2_mean)

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file'.format(
      name=fn, length=len(uploaded[fn])))

path='/content/'

files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
x=[]
y=[]
n=[]
l=["A", "E", "I", "O", "U", "Α", "Ε", "Ι", "Ο", "Υ"]

for f in files:
    filename = Path(f).stem
    for i in l:
      if i in filename:
        f1,f2=measureFormants(100, 2000, f)
        x.append(f1)
        y.append(f2)
        n.append('   '+filename)
volume = 20
fig, ax = plt.subplots()
ax.scatter(y, x, s=volume, alpha=0.5, color='purple')
plt.gca().invert_xaxis()
plt.gca().invert_yaxis()
plt.xlabel("F2")
plt.ylabel("F1")
plt.title("VOWELMAP")
for i, txt in enumerate(n):
    ax.annotate(txt, (y[i], x[i]))


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Phonetogram

In [ ]:
# Assistance for students in Singing and Music Aesthetics (ASMA) 2022
# Version 2
# Research Team:
# Anastasia Georgaki (Coordinator)
# Areti Andreopoulou, Iakovos Steinhauer, Vangelis Angelakis, Sofia Stavropoulou, Natalia Kotsani, Giorgos Dedousis, Kostas Katsantonis
# Author: Natalia Kotsani

!pip install praat-parselmouth
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import struct
import parselmouth
from scipy import signal as sg
from scipy.interpolate import interpolate
from scipy.interpolate import interp1d
import os
from pathlib import Path
import sys
from matplotlib import ticker

sns.set()
path='/content/'
files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
for f in files:
  if not f.startswith('.'):
    filename = Path(f).stem
    snd = parselmouth.Sound(path+filename+'.wav')
    pitch = snd.to_pitch(pitch_floor=75.0, pitch_ceiling=600.00, time_step=0.01)
    intensity = snd.to_intensity(subtract_mean=True, minimum_pitch=75.0 * 2, time_step = 0.01)
    pitch_values = pitch.selected_array['frequency']
    pitch_values[pitch_values==0] = np.nan
    intensity_values = (intensity.values).flatten()
    if (pitch_values.size<intensity_values.size):
       pitch_values = np.append(pitch_values, pitch_values[pitch_values.size-1])
    if (pitch_values.size>intensity_values.size):
       intensity_values = np.append(intensity_values, intensity_values[intensity_values.size-1])
    plt.plot(pitch_values, intensity_values, 'o', markersize=5, color='purple' , alpha=0.3)

x = np.array([110.00, 220.00, 440.00, 880.00])
my_xticks = ['A2','A3', 'A4','A5']
plt.xscale('log')
ax = plt.gca()
ax.xaxis.set_minor_locator(ticker.NullLocator())
ax.xaxis.set_major_formatter(ticker.ScalarFormatter())
plt.grid(False)
plt.xlabel("Fundamental Frequency [Hz]")
plt.ylabel("Intensity [dB]")
plt.grid(True)
plt.xticks(x, my_xticks)
plt.title("PHONETOGRAM")
plt.savefig('VRP')

# Breathiness

# Nasality